In [ ]:
# General imports
import qiskit 
from qiskit_nature.second_q.formats import MoleculeInfo
from qiskit_nature.units import DistanceUnit

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms.minimum_eigensolvers import VQE

from qiskit import Aer

# Plotting functions
import matplotlib.pyplot as plt

backend = Aer.get_backend('aer_simulator')

In [ ]:
def callback(eval_count, parameters, mean, std):
    callback_dict["iters"].append(eval_count) 
    callback_dict["prev_vector"] = parameters
    callback_dict["cost_history"].append(mean)
    current_time = time.perf_counter()
    if callback_dict["iters"][-1] > 1:
        callback_dict["_total_time"] += current_time - callback_dict["_prev_time"]
        callback_dict['iteration_time'].append(current_time - callback_dict["_prev_time"])
    callback_dict["_prev_time"] = current_time
    time_str = (round(callback_dict["_total_time"] / (callback_dict["iters"][-1] - 1), 2)
        if callback_dict["_total_time"]
        else "-")
    print("Iters. done: {} [Avg. time per iter: {}]".format(callback_dict["iters"][-1], time_str),
        end="\r", flush=True,)       

In [ ]:
callback_dict = {
    "prev_vector": None,
    "iters": [],
    "cost_history": [],
    'iteration_time': [],
    "_total_time": 0,
    "_prev_time": 0,
}

In [ ]:
molecule = MoleculeInfo(['H','H'],[[0.74, 0.0, 0.0],[0.0, 0.0, 0.0]] , 
                        multiplicity=1, 
                        charge=0, 
                        units=DistanceUnit.ANGSTROM)
driver = PySCFDriver.from_molecule(molecule ,basis= '3-21G')
system = driver.run()
mapper = JordanWignerMapper()
algo = NumPyMinimumEigensolver()
algo.filter_criterion = system.get_default_filter_criterion()
solver = GroundStateEigensolver(mapper, algo)
result_classic = solver.solve(system)
initial_point = HartreeFock(system.num_spatial_orbitals, system.num_particles, mapper)
ansatz = UCCSD(system.num_spatial_orbitals, 
               system.num_particles,
               mapper,
               initial_state=initial_point)
estimator = Estimator(options={'shots':800})
vqe = VQE(ansatz=ansatz, estimator=estimator, optimizer=COBYLA())
hamiltonian = mapper.map(system.hamiltonian.second_q_op())
vqe_result = vqe.compute_minimum_eigenvalue(operator=hamiltonian)
energia_total_vqe = vqe_result.eigenvalue.real+result_classic.nuclear_repulsion_energy

In [ ]:
print('O resultado para energia total clássica é {}'.format(result_classic.total_energies[0]))
print('O resultado para energia total quantica é {}'.format(energia_total_vqe))
difference = (result_classic.total_energies[0]-energia_total_vqe)*2625.5
print('A diferença entre os dois pontos é de: ΔE = {} kJ/mol'.format(difference))